In [ ]:
#@title License information
# Copyright 2023 The MediaPipe Authors.
# Licensed under the Apache License, Version 2.0 (the "License");
#
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

The MediaPipe Model Maker package is a low-code solution for customizing on-device machine learning (ML) Models.

This notebook shows the end-to-end process of customizing a gesture recognizer model for recognizing some common hand gestures in the [HaGRID](https://www.kaggle.com/datasets/innominate817/hagrid-sample-30k-384p) dataset.

## Prerequisites

Install the MediaPipe Model Maker package.

In [1]:
!pip install --upgrade pip
!pip install mediapipe-model-maker
!pip install opendatasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 933.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-metadata to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

Import the required libraries.

In [2]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
import opendatasets as od

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Simple End-to-End Example

This end-to-end example uses Model Maker to customize a model for on-device gesture recognition.

### Get the dataset

The dataset for gesture recognition in model maker requires the following format: `<dataset_path>/<label_name>/<img_name>.*`. In addition, one of the label names (`label_names`) must be `none`. The `none` label represents any gesture that isn't classified as one of the other gestures.

This example uses a rock paper scissors dataset sample which is downloaded from GCS.

In [3]:
od.download(
    "https://www.kaggle.com/datasets/kapillondhe/american-sign-language")
train_dataset_path = "/content/american-sign-language/ASL_Dataset/Train/"
test_dataset_path = "/content/american-sign-language/ASL_Dataset/Test/"

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: avitejiyer
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/kapillondhe/american-sign-language


100%|██████████| 4.64G/4.64G [00:35<00:00, 139MB/s]


In [5]:
def rename_folder(root_dir, old_name="Nothing", new_name="None"):
    """
    Rename a folder from `old_name` to `new_name` within `root_dir`.

    Parameters:
    root_dir (str): The root directory containing the folder to rename.
    old_name (str): The current name of the folder to rename.
    new_name (str): The new name for the folder.
    """
    # Construct the full path for the old and new folder names
    old_folder_path = os.path.join(root_dir, old_name)
    new_folder_path = os.path.join(root_dir, new_name)

    # Check if the folder exists
    if os.path.exists(old_folder_path) and os.path.isdir(old_folder_path):
        # Rename the folder
        os.rename(old_folder_path, new_folder_path)
        print(f"Renamed folder from '{old_name}' to '{new_name}'")
    else:
        print(f"Folder '{old_name}' does not exist in the directory '{root_dir}'")


rename_folder(train_dataset_path)
rename_folder(test_dataset_path)

Renamed folder from 'Nothing' to 'None'
Renamed folder from 'Nothing' to 'None'


Trimming the train dataset for faster processing - change once model is verified.

In [4]:
import shutil

def delete_excess_images(root_dir, max_images=500):
    """
    Delete all but `max_images` images in each subdirectory of `root_dir`.

    Parameters:
    root_dir (str): The root directory containing subdirectories of images.
    max_images (int): The maximum number of images to keep in each subdirectory.
    """
    # List all subdirectories in the root directory
    for subdir, _, files in os.walk(root_dir):
        # Filter out files that are not images (e.g., only keep jpg, png files)
        image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # If there are more images than the max allowed, delete the excess
        if len(image_files) > max_images:
            # Sort the list of images to ensure we keep the first 500
            image_files.sort()
            # Get the images to delete
            images_to_delete = image_files[max_images:]
            # Delete each excess image
            for image in images_to_delete:
                image_path = os.path.join(subdir, image)
                os.remove(image_path)
                print(f"Deleted {image_path}")

delete_excess_images(train_dataset_path)


Streaming output truncated to the last 5000 lines.
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1637.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1638.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1639.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1640.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1641.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1642.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1643.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1644.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1645.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1646.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1647.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1648.jpg
Deleted /content/american-sign-language/ASL_Dataset/Train/T/1649.jpg
Deleted /content/american-sign-language/ASL_Dataset/

Verify the rock paper scissors dataset by printing the labels. There should be 4 gesture labels, with one of them being the `none` gesture.

In [8]:
print(train_dataset_path)
labels = []
for i in os.listdir(train_dataset_path):
  if os.path.isdir(os.path.join(train_dataset_path, i)):
    labels.append(i)
print(labels)

/content/american-sign-language/ASL_Dataset/Train/
['U', 'L', 'Y', 'W', 'C', 'R', 'G', 'S', 'D', 'V', 'H', 'A', 'Space', 'N', 'B', 'O', 'X', 'P', 'J', 'K', 'Z', 'F', 'None', 'M', 'Q', 'I', 'E', 'T']


To better understand the dataset, plot a couple of example images for each gesture.

In [10]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(train_dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

<ipython-input-10-8465a736b382>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))


### Run the example
The workflow consists of 4 steps which have been separated into their own code blocks.

**Load the dataset**

Load the dataset located at `dataset_path` by using the `Dataset.from_folder` method. When loading the dataset, run the pre-packaged hand detection model from MediaPipe Hands to detect the hand landmarks from the images. Any images without detected hands are ommitted from the dataset. The resulting dataset will contain the extracted hand landmark positions from each image, rather than images themselves.

The `HandDataPreprocessingParams` class contains two configurable options for the data loading process:
* `shuffle`: A boolean controlling whether to shuffle the dataset. Defaults to true.
* `min_detection_confidence`: A float between 0 and 1 controlling the confidence threshold for hand detection.

Split the dataset: 80% for training, 10% for validation, and 10% for testing.

In [12]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=train_dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, validation_data = data.split(0.9)

test_data = gesture_recognizer.Dataset.from_folder(
    dirname=test_dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)


Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder


**Train the model**

Train the custom gesture recognizer by using the create method and passing in the training data, validation data, model options, and hyperparameters. For more information on model options and hyperparameters, see the [Hyperparameters](#hyperparameters) section below.

In [13]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 28)                3612      
 out (Dense)                                                     
                                                             

**Evaluate the model performance**

After training the model, evaluate it on a test dataset and print the loss and accuracy metrics.

In [14]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

99/99 [==============================] - 1s 2ms/step - loss: 0.1355 - categorical_accuracy: 0.9192
Test loss:0.13552120327949524, Test accuracy:0.9191918969154358


**Export to Tensorflow Lite Model**

After creating the model, convert and export it to a Tensorflow Lite model format for later use on an on-device application. The export also includes model metadata, which includes the label file.

In [15]:
model.export_model()
!ls exported_model

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
best_model_weights.data-00000-of-00001	checkpoint    gesture_recognizer.task  metadata.json
best_model_weights.index		epoch_models  logs


In [16]:
files.download('exported_model/gesture_recognizer.task')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Run the model on-device

To use the TFLite model for on-device usage through MediaPipe Tasks, refer to the Gesture Recognizer [overview page](https://developers.google.com/mediapipe/solutions/vision/gesture_recognizer).

## Hyperparameters {:#hyperparameters}


You can further customize the model using the `GestureRecognizerOptions` class, which has two optional parameters for `ModelOptions` and `HParams`. Use the `ModelOptions` class to customize parameters related to the model itself, and the `HParams` class to customize other parameters related to training and saving the model.

`ModelOptions` has one customizable parameter that affects accuracy:
* `dropout_rate`: The fraction of the input units to drop. Used in dropout layer. Defaults to 0.05.
* `layer_widths`: A list of hidden layer widths for the gesture model. Each element in the list will create a new hidden layer with the specified width. The hidden layers are separated with BatchNorm, Dropout, and ReLU. Defaults to an empty list(no hidden layers).

`HParams` has the following list of customizable parameters which affect model accuracy:
* `learning_rate`: The learning rate to use for gradient descent training. Defaults to 0.001.
* `batch_size`: Batch size for training. Defaults to 2.
* `epochs`: Number of training iterations over the dataset. Defaults to 10.
* `steps_per_epoch`: An optional integer that indicates the number of training steps per epoch. If not set, the training pipeline calculates the default steps per epoch as the training dataset size divided by batch size.
* `shuffle`: True if the dataset is shuffled before training. Defaults to False.
* `lr_decay`: Learning rate decay to use for gradient descent training. Defaults to 0.99.
* `gamma`: Gamma parameter for focal loss. Defaults to 2

Additional `HParams` parameter that does not affect model accuracy:
* `export_dir`: The location of the model checkpoint files and exported model files.

For example, the following trains a new model with the dropout_rate of 0.2 and learning rate of 0.003.

In [ ]:
hparams = gesture_recognizer.HParams(learning_rate=0.003, export_dir="exported_model_2")
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.2)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model_2 = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Evaluate the newly trained model.

In [ ]:
loss, accuracy = model_2.evaluate(test_data)
print(f"Test loss:{loss}, Test accuracy:{accuracy}")